In [4]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time

## Обязательная часть

    Вам необходимо написать функцию, которая будет основана на поиске по сайту habr.com. Функция в качестве параметра должна принимать список запросов для поиска (например, ['python', 'анализ данных']) и на основе материалов, попавших в результаты поиска по каждому запросу, возвращать датафрейм вида:

<дата> - <заголовок> - <ссылка на материал>

    В рамках задания предполагается работа только с одной (первой) страницей результатов поисковой выдачи для каждого запроса. Материалы в датафрейме не должны дублироваться, если они попадали в результаты поиска для нескольких запросов из списка.


In [21]:
url = 'https://habr.com/ru/search/'    
params = {
    'page': 1,
    'q': 'python'
    }
res = requests.get(url, params=params)
soup = BeautifulSoup(res.text) 
soup

<!DOCTYPE html>
<html data-vue-meta="%7B%22lang%22:%7B%22ssr%22:%22ru%22%7D%7D" lang="ru">
<head>
<meta charset="utf-8"/>
<meta content="width=device-width,initial-scale=1.0,viewport-fit=cover" name="viewport"/>
<meta content="unsafe-url" name="referrer"/>
<title>Результаты поиска по запросу «python» / Хабр</title>
<style>
    /* cyrillic-ext */
    @font-face {
      font-family: 'Fira Sans';
      font-style: normal;
      font-weight: 500;
      font-display: swap;
      src: url(https://fonts.gstatic.com/s/firasans/v11/va9B4kDNxMZdWfMOD5VnZKveSxf6TF0.woff2) format('woff2');
      unicode-range: U+0460-052F, U+1C80-1C88, U+20B4, U+2DE0-2DFF, U+A640-A69F, U+FE2E-FE2F;
    }

    /* cyrillic */
    @font-face {
      font-family: 'Fira Sans';
      font-style: normal;
      font-weight: 500;
      font-display: swap;
      src: url(https://fonts.gstatic.com/s/firasans/v11/va9B4kDNxMZdWfMOD5VnZKveQhf6TF0.woff2) format('woff2');
      unicode-range: U+0400-045F, U+0490-0491, U+04B0-04B1

In [23]:
publications = soup.find_all('article', class_='tm-articles-list__item')
len(publications)

20

In [76]:
for post in publications:
    date = post.find('time').text
    print(date)
    if post.find('h2', 'tm-article-snippet__title tm-article-snippet__title_h2'): #Так как присутствует всратый вид постов нужно сделать альтернативное получение заголовка и ссылки
        heading = post.find('h2', 'tm-article-snippet__title tm-article-snippet__title_h2').text
    else:
        heading = post.find('h2', 'tm-megapost-snippet__title').text 
    print(heading)
    if post.find('a', 'tm-article-snippet__title-link'):
        link = url[:-11] + post.find('a', 'tm-article-snippet__title-link').get('href') #атрибут содержит не прямую ссылку, пришлось предусмотреть создание полной рабочей сслыки
    else:
        link = url[:-11] + post.find('a', 'tm-megapost-snippet__link tm-megapost-snippet__date').get('href')
    print(link)

25  июля  2012 в 13:29
Пять лет Школе анализа данных
https://habr.com/ru/company/yandex/blog/148443/
25  июля  2017 в 13:47
10 лет Школе анализа данных Яндекса
https://habr.com/ru/company/yandex/blog/334066/
14  января  2021 в 10:00
Что такое бессерверный SQL? И как использовать его для анализа данных?
https://habr.com/ru/company/microsoft/blog/537064/
21  января  2019 в 11:21
Как мы строим систему обработки, хранения и анализа данных в СИБУРе
https://habr.com/ru/company/sibur_official/blog/436632/
14  мая  2020 в 13:39
Ubic создаст платформу для анализа данных о москвичах
https://habr.com/ru/news/t/501944/
16  апреля  2020 в 07:10
Microsoft анонсировала запуск «Планетарного компьютера» для сбора, хранения и анализа данных о состоянии Земли
https://habr.com/ru/news/t/497474/
13  ноября  2021 в 19:15
Владимир Путин выступил с Германом Грефом на конференции по ИИ и анализу данных Artificial Intelligence Journey 2021
https://habr.com/ru/news/t/588991/
10  декабря  2021 в 13:15
Интенсив дл

In [74]:
def searching_in_habr(list_of_search):
    """Поиск на сайте Хабр по запросам в списке поданным на вход и формарование
    датафрейма с постами и информацией о них удовлетворяющие резултатам запроса"""
    
    resault_of_search = pd.DataFrame()
    url = 'https://habr.com/ru/search/' 
    
    for search in list_of_search:
        params = {'q': search}
        res = requests.get(url, params=params)
        soup = BeautifulSoup(res.text) 
        publications = soup.find_all('article', class_='tm-articles-list__item')
        for post in publications:
            date = post.find('time').text
            if post.find('h2', 'tm-article-snippet__title tm-article-snippet__title_h2'):
                heading = post.find('h2', 'tm-article-snippet__title tm-article-snippet__title_h2').text
            else:
                heading = post.find('h2', 'tm-megapost-snippet__title').text
            if post.find('a', 'tm-article-snippet__title-link'):
                link = url[:-11] + post.find('a', 'tm-article-snippet__title-link').get('href')
            else:
                link = url[:-11] + post.find('a', 'tm-megapost-snippet__link tm-megapost-snippet__date').get('href')
            row = {'date': date, 'title': heading, 'link': link}
            resault_of_search = pd.concat([resault_of_search, pd.DataFrame([row])]) 
    return resault_of_search.reset_index(drop=True).drop_duplicates()

In [48]:
list_of_search = ['python', 'анализ данных']

In [77]:
searching_in_habr(list_of_search)

,date,title,link
0,13 декабря 2021 в 09:00,Жаждущим автоматизации: открытый урок «ChatOps...,https://habr.com/ru/company/southbridge/news/t...
1,20 января в 18:37,Курс «Python для инженеров». Старт 3 потока 31...,https://habr.com/ru/company/southbridge/news/t...
2,21 апреля 2020 в 18:35,"Вышел Python 2.7.18, последний релиз ветки Pyt...",https://habr.com/ru/news/t/498364/
3,6 июля 2021 в 13:29,Python Community Meetup 8/07: видео и материал...,https://habr.com/ru/company/raiffeisenbank/new...
4,13 января в 18:35,Открытый урок «Пишем Custom Prometheus Exporte...,https://habr.com/ru/company/southbridge/news/t...
5,4 декабря 2020 в 21:03,Python как компилируемый статически типизирова...,https://habr.com/ru/news/t/531402/
6,8 марта в 12:13,Вышел мартовский релиз расширения Python для V...,https://habr.com/ru/news/t/654707/
7,3 марта 2020 в 13:22,... начале этого года Python сместил Java и ...,https://habr.com/ru/company/itsumma/news/t/490...
8,16 ноября 2021 в 16:09,EPAM разработала бесплатный курс по программир...,https://habr.com/ru/company/epam_systems/news/...
9,3 июня 2020 в 09:00,Вышла версия 0.0.2 snakeware — дистрибутива Li...,https://habr.com/ru/news/t/505096/


## Уф, переходим к дополнительному заданию
## Дополнительная часть (необязательная)

Функция из обязательной части задания должна быть расширена следующим образом:

     кроме списка ключевых слов для поиска необходимо объявить параметр с количеством страниц поисковой выдачи. Т.е. при передаче в функцию аргумента 4 необходимо получить материалы с первых 4 страниц результатов;
    в датафрейме должны быть столбцы с полным текстом найденных материалов и количеством лайков:

    <дата> - <заголовок> - <ссылка на материал> - <текст материала> - <количество лайков>



In [91]:
list_of_search = ['python', 'анализ данных']
number_of_pages = 1

In [89]:
def searching_in_habr(list_of_search, number_of_pages):
    """Поиск на сайте Хабр по списку запросов поданным на вход и формарование
    датафрейма с постами и информацией о них, удовлетворяющие результатам запроса"""
    
    resault_of_search = pd.DataFrame()
    url = 'https://habr.com/ru/search/' 
    
    for search in list_of_search:
        params = {'q': search}
        for page in range(1, number_of_pages+1):
            params['page'] = page
            res = requests.get(url, params=params)
            soup = BeautifulSoup(res.text) 
            publications = soup.find_all('article', class_='tm-articles-list__item')
            for post in publications:
                date = post.find('time').text
                if post.find('h2', 'tm-article-snippet__title tm-article-snippet__title_h2'):
                    heading = post.find('h2', 'tm-article-snippet__title tm-article-snippet__title_h2').text
                else:
                    heading = post.find('h2', 'tm-megapost-snippet__title').text
                if post.find('a', 'tm-article-snippet__title-link'):
                    link = url[:-11] + post.find('a', 'tm-article-snippet__title-link').get('href')
                else:
                    link = url[:-11] + post.find('a', 'tm-megapost-snippet__link tm-megapost-snippet__date').get('href')
                if post.find('span', 'tm-votes-meter__value tm-votes-meter__value_positive tm-votes-meter__value_appearance-article tm-votes-meter__value_rating'):
                    likes = post.find('span', 'tm-votes-meter__value tm-votes-meter__value_positive tm-votes-meter__value_appearance-article tm-votes-meter__value_rating').text
                else:
                    likes = ''
                
                req_text = requests.get(link).text #открываем по ссылке каждый пост и достаем полный текст
                soup_text = BeautifulSoup(req_text)
                if soup_text.find('div', class_='article-formatted-body article-formatted-body_version-1'):
                    post_text = soup_text.find('div', class_='article-formatted-body article-formatted-body_version-1').text.strip()
                else:
                    post_text = 'out of standart'
                row = {'date': date, 'title': heading, 'link': link, 'post_text': post_text, 'likes': likes}
                resault_of_search = pd.concat([resault_of_search, pd.DataFrame([row])]) 
    return resault_of_search.reset_index(drop=True).drop_duplicates()

In [92]:
searching_in_habr(list_of_search, number_of_pages)

,date,title,link,post_text,likes
0,13 декабря 2021 в 09:00,Жаждущим автоматизации: открытый урок «ChatOps...,https://habr.com/ru/company/southbridge/news/t...,out of standart,+9
1,20 января в 18:37,Курс «Python для инженеров». Старт 3 потока 31...,https://habr.com/ru/company/southbridge/news/t...,out of standart,+10
2,21 апреля 2020 в 18:35,"Вышел Python 2.7.18, последний релиз ветки Pyt...",https://habr.com/ru/news/t/498364/,"20 апреля 2020 года, спустя почти десять лет п...",+19
3,6 июля 2021 в 13:29,Python Community Meetup 8/07: видео и материал...,https://habr.com/ru/company/raiffeisenbank/new...,out of standart,+3
4,13 января в 18:35,Открытый урок «Пишем Custom Prometheus Exporte...,https://habr.com/ru/company/southbridge/news/t...,out of standart,+10
5,4 декабря 2020 в 21:03,Python как компилируемый статически типизирова...,https://habr.com/ru/news/t/531402/,out of standart,+16
6,8 марта в 12:13,Вышел мартовский релиз расширения Python для V...,https://habr.com/ru/news/t/654707/,out of standart,
7,3 марта 2020 в 13:22,... начале этого года Python сместил Java и ...,https://habr.com/ru/company/itsumma/news/t/490...,"Согласно отчету RedMonk за январь 2020 года, P...",+31
8,16 ноября 2021 в 16:09,EPAM разработала бесплатный курс по программир...,https://habr.com/ru/company/epam_systems/news/...,out of standart,+3
9,3 июня 2020 в 09:00,Вышла версия 0.0.2 snakeware — дистрибутива Li...,https://habr.com/ru/news/t/505096/,31 мая 2020 года разработчик Джош Мур (Josh Mo...,+19
